In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

## Marina's import code

In [ ]:
#import 2019 data
CV2019_SQ = pd.read_csv('/Users/Marina/Desktop/CV_REE_working/LA_ICPMS_data/2019_Rutgers/2019_python_input/2019_LA_CVdata_SQ_input.csv',index_col=None)
CV2019_SQ = CV2019_SQ.fillna(np.nan)
CV2019_SQ.Spot_Size = CV2019_SQ.Spot_Size.astype(int)
CV2019_SQ.Spot_Num = CV2019_SQ.Spot_Num.astype(int)
CV2019_SQ.Standard_Spot_Size = CV2019_SQ.Standard_Spot_Size.astype(int)

CV2019_Si = pd.read_csv('/Users/Marina/Desktop/CV_REE_working/LA_ICPMS_data/2019_Rutgers/2019_python_input/2019_LA_CVdata_Si_input.csv',index_col=None)
CV2019_Si = CV2019_Si.fillna(np.nan)
CV2019_Si.Spot_Size = CV2019_Si.Spot_Size.astype(int)
CV2019_Si.Spot_Num = CV2019_Si.Spot_Num.astype(int)
CV2019_Si.Standard_Spot_Size = CV2019_Si.Standard_Spot_Size.astype(int)

CV2019_Mg = pd.read_csv('/Users/Marina/Desktop/CV_REE_working/LA_ICPMS_data/2019_Rutgers/2019_python_input/2019_LA_CVdata_Mg_input.csv',index_col=None)
CV2019_Mg = CV2019_Mg.fillna(np.nan)
CV2019_Mg.Spot_Size = CV2019_Mg.Spot_Size.astype(int)
CV2019_Mg.Spot_Num = CV2019_Mg.Spot_Num.astype(int)
CV2019_Mg.Standard_Spot_Size = CV2019_Mg.Standard_Spot_Size.astype(int)

CV2019_Ca = pd.read_csv('/Users/Marina/Desktop/CV_REE_working/LA_ICPMS_data/2019_Rutgers/2019_python_input/2019_LA_CVdata_Ca_input.csv',index_col=None)
CV2019_Ca = CV2019_Ca.fillna(np.nan)
CV2019_Ca.Spot_Size = CV2019_Ca.Spot_Size.astype(int)
CV2019_Ca.Spot_Num = CV2019_Ca.Spot_Num.astype(int)
CV2019_Ca.Standard_Spot_Size = CV2019_Ca.Standard_Spot_Size.astype(int)


#import 2022 data
CV2022_SQ = pd.read_csv('/Users/Marina/Desktop/2022_REE_work/Rutgers_June2022/Rutgers_data_June2022/2022_python_input/2022_June_SQ_input2.csv',index_col=None)
CV2022_SQ = CV2022_SQ.fillna(np.nan)
CV2022_SQ.Spot_Size = CV2022_SQ.Spot_Size.astype(int)
CV2022_SQ.Spot_Num = CV2022_SQ.Spot_Num.astype(int)
CV2022_SQ.Standard_Spot_Size = CV2022_SQ.Standard_Spot_Size.astype(int)
CV2022_SQ.Met_Name = CV2022_SQ.Met_Name.astype(str)
CV2022_SQ.Met_Type = CV2022_SQ.Met_Type.astype(str)

CV2022_Si = pd.read_csv('/Users/Marina/Desktop/2022_REE_work/Rutgers_June2022/Rutgers_data_June2022/2022_python_input/2022_June_Si_input2.csv',index_col=None)
CV2022_Si = CV2022_Si.fillna(np.nan)
CV2022_Si.Spot_Size = CV2022_Si.Spot_Size.astype(int)
CV2022_Si.Spot_Num = CV2022_Si.Spot_Num.astype(int)
CV2022_Si.Standard_Spot_Size = CV2022_Si.Standard_Spot_Size.astype(int)

CV2022_Mg = pd.read_csv('/Users/Marina/Desktop/2022_REE_work/Rutgers_June2022/Rutgers_data_June2022/2022_python_input/2022_June_Mg_input2.csv',index_col=None)
CV2022_Mg = CV2022_Mg.fillna(np.nan)
CV2022_Mg.Spot_Size = CV2022_Mg.Spot_Size.astype(int)
CV2022_Mg.Spot_Num = CV2022_Mg.Spot_Num.astype(int)
CV2022_Mg.Standard_Spot_Size = CV2022_Mg.Standard_Spot_Size.astype(int)

CV2022_Ca = pd.read_csv('/Users/Marina/Desktop/2022_REE_work/Rutgers_June2022/Rutgers_data_June2022/2022_python_input/2022_June_Ca_input2.csv',index_col=None)
CV2022_Ca = CV2022_Ca.fillna(np.nan)
CV2022_Ca.Spot_Size = CV2022_Ca.Spot_Size.astype(int)
CV2022_Ca.Spot_Num = CV2022_Ca.Spot_Num.astype(int)
CV2022_Ca.Standard_Spot_Size = CV2022_Ca.Standard_Spot_Size.astype(int)

CV2022_Al = pd.read_csv('/Users/Marina/Desktop/2022_REE_work/Rutgers_June2022/Rutgers_data_June2022/2022_python_input/2022_June_Al_input2.csv',index_col=None)
CV2022_Al = CV2022_Al.fillna(np.nan)
CV2022_Al.Spot_Size = CV2022_Al.Spot_Size.astype(int)
CV2022_Al.Spot_Num = CV2022_Al.Spot_Num.astype(int)
CV2022_Al.Standard_Spot_Size = CV2022_Al.Standard_Spot_Size.astype(int)
